In [1]:
import os
import requests
import re
from bs4 import BeautifulSoup
import openpyxl

In [2]:
def hasdigit(url):
    return any(char.isdigit() for char in url)

In [3]:
def HavaAlphabet(url):
    return any(char.isalpha() for char in url)

In [4]:
def NewYorkTimesUrlFilter(url):
    feature1 = url[:23]
#     feature2 = url[-10:]
    if feature1 != 'https://www.nytimes.com':
        return False
#     elif HavaAlphabet(feature2):
#         return False
    else:
        return True

In [5]:
nytimes = 'https://www.nytimes.com/section/business'

In [6]:
saving_path = '.\\7_websites_output'
if not os.path.exists(saving_path):
    os.makedirs(saving_path)

In [7]:
NYT_homePage = "#site-content"
NYT_NewsPage = ".css-axufdj.evys1bk0"
NYT_NewsPage_title1 = '.css-1vkm6nb.ehdk2mb0'
NYT_NewsPage_title2 = '.css-16owfwv.e1h9rw200'

In [8]:
def ft_NYT():
    prefix = 'https://www.nytimes.com'
    r = requests.get(nytimes)

    dateRE = re.compile("(19|20)\d\d[- /.](0[1-9]|1[012])[- /.](0[0-9]|[12][0-9]|3[01])")

    if r.status_code == 200:  # print(r.status_code)
        soup = BeautifulSoup(r.text, 'html.parser')
        result = soup.select(NYT_homePage)

    raw_url_list = []
    target_url_list = []
    for results in result:
        class_a = results.find_all("a")

        for i in class_a:
            if i['href'][0] == '/':
                raw_url_list.append(str(prefix+i['href']))
            else:
                raw_url_list.append(str(i['href']))

        for url in raw_url_list:
            if NewYorkTimesUrlFilter(url):
                if url in target_url_list:
                    pass
                else:
                    if re.search(dateRE, url):
                        target_url_list.append(url)

    workbook = openpyxl.Workbook()
    sheet = workbook.active

    ctr = 1
    for url in target_url_list[:]:
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            result = soup.select(NYT_NewsPage)
            title_select = soup.select(NYT_NewsPage_title1)
            if title_select == []:
                title_select = soup.select(NYT_NewsPage_title2)

            title = ''
            for i in title_select:
                    title = i.text

            # print("-----------------------------------------------------------------------------------------------")
            # print(title)
            # print(url)
            sheet['A'+str(ctr)] = title
            sheet['B'+str(ctr)] = url

            content = []
            for results in result:
                content.append(results.text)

            if content == []:
                print("Can't crawler the web")
                print(url)
                sheet['C'+str(ctr)] = "Can't Crawler the Web  (Maybe got banned or pay to watch)"
                sheet['D'+str(ctr)] = ''
                ctr += 1
            else:
                for sentence in content:
                    sheet['D'+str(ctr)] = str(sentence)
    #                 print(sentence)
                    ctr += 1

    workbook.save(saving_path+'\\NYtimes.xlsx')
    
    print('finish')

In [9]:
ft_NYT()

finish
